## Source

This notebook contains code from: https://github.com/harvardnlp/annotated-transformer

Original license: 

> MIT License
> 
> Copyright (c) 2018 Alexander Rush
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
> 
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
> 
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.


## License

The new work is licensed under the BSD 3-Clause License.


Copyright (c) 2018, Guillaume Chevalier

All rights reserved.


In [1]:

from data.read_write_txt import FilesReaderBinaryUTF8
from model.sgnn_projection_layer import get_sgnn_projection_pipeline


In [2]:
import math
import copy
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt


In [3]:
class Projector(nn.Module):
    "Define Matching Network projection comparison step."
    def __init__(self, d_model, vocab):
        super(Projector, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)


class ProjectEncode(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, sgnn_projector, encoder):
        super(ProjectEncode, self).__init__()
        self.sgnn_projector = sgnn_projector
        self.encoder = encoder
        
    def forward(self, x, mask):
        "Take in and process sequences, using word projector and encoder."
        x_projected = self.sgnn_projector(x)
        return self.encoder(x_projected, mask)


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N_encoder_layers):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N_encoder_layers)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + sublayer(self.norm(x))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

def attention(query, key, value, mask=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        
    def forward(self, query, key, value, mask):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.w_2(F.relu(self.w_1(x)))


class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return x


class SGNNWordProjections(nn.Module):
    def __init__(self, d_model, T_sgnn=80, d_sgnn=14):
        super(SGNNWordProjections, self).__init__()
        self.sgnn_pipeline = get_sgnn_projection_pipeline(T=T_sgnn, d=d_sgnn)
        self.d_model = d_model
        self.w = nn.Linear(T_sgnn * d_sgnn, d_model)

    def forward(self, x):
        i = self.sgnn_pipeline.transform(x)
        i = torch.FloatTensor(i)
        h = self.w(i)
        lrelu = F.relu(h) + h / 5.0
        return lrelu

In [4]:
def make_model(N_encoder_layers=6, d_model=512, d_ff=2048, h=8, T_sgnn=80, d_sgnn=14):
    """Helper: Construct a model from hyperparameters."""
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff)
    position = PositionalEncoding(d_model)
    
    model = ProjectEncode(
        nn.Sequential(SGNNWordProjections(d_model, T_sgnn, d_sgnn), c(position)),
        Encoder(EncoderLayer(d_model, c(attn), c(ff)), N_encoder_layers)
    )
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return model


tmp_model = make_model(d_ff=1024)

In [6]:
list_of_sentences = [
    "test 1234 wer wer wer werqwerqwereqrw edfwewf efw",
    "fgerw fewfew iuosdfjohi",
    "dsfjoi dsf dsf dfs dfs asd apwdoijas pdoija spdokj aspodkj aspdojk aopsidj SAUOIEDFh zsilkudfh zslkudf h iaushdfoiuoh asdfiouh adsfiouh adfsihou adsfdfsiohu fsadihou fdsa"
]*5

sentence_projection = tmp_model(list_of_sentences)
sentence_projection.shape

TypeError: forward() missing 1 required positional argument: 'mask'

In [25]:
F.max_pool1d(sentence_projection, kernel_size=sentence_projection.size()[-1]).shape

torch.Size([10, 53, 1])


https://github.com/harvardnlp/annotated-transformer/blob/master/The%20Annotated%20Transformer.ipynb :
> The maximum batchsize does not surpass a threshold (25000 if we have 8 gpus)

> We trained the base models for a total of 100,000 steps or 12 hours. For our big models, step time was 1.0 seconds. The big models were trained for 300,000 steps (3.5 days).



In [7]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.ntokens = (self.trg_y != pad).data.sum()


def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        raise ValueError("TODO: make target zeros and ones, whereas zeros must not fit, and ones must fit")
        out = model.forward(batch.src, batch.src_mask)  # TODO: batch.trg argument here.
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / float(total_tokens)


global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)


class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))


def loss(x):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d],
                                 ])
    #print(predict)
    return crit(Variable(predict.log()),
                 Variable(torch.LongTensor([1]))).data[0]


def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)

class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, projector, criterion, opt=None):
        self.projector = projector
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.projector(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data[0] * norm


In [29]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)

In [40]:
c = data_gen(1000, 32, 777).__iter__().__next__()

In [41]:
c.src_mask.shape, c.src.shape, c.src

(torch.Size([32, 1, 10]),
 torch.Size([32, 10]),
 tensor([[   1,   62,  889,  578,  774,  101,  629,  937,  496,  950],
         [   1,  275,  494,   34,  810,  329,  803,   30,  328,   41],
         [   1,   53,   64,  250,  225,  462,  794,  505,  796,  339],
         [   1,   72,  903,  465,  924,  265,  805,  874,  723,  799],
         [   1,  683,   70,  416,  353,  494,  922,   67,  257,  380],
         [   1,  796,  145,  611,  874,   86,  603,   37,  606,  131],
         [   1,  655,  244,  705,  746,  384,  450,  170,  995,  434],
         [   1,   83,  604,  182,  969,  563,   46,  420,  253,  303],
         [   1,  771,  746,  346,  580,  780,  836,  233,    8,  623],
         [   1,  355,  769,  855,  972,  442,  802,  481,  599,  574],
         [   1,  793,  513,  312,  133,  938,  237,  355,  454,  372],
         [   1,  623,  171,  405,  770,  964,  435,  891,  502,  776],
         [   1,   95,  755,  836,  985,  942,  619,  146,  453,  379],
         [   1,   75,  534, 